# Lab 02. LGU+ 2021 Summer - Content-based Recommendation 실습

## Table of Contents
- Content-based Recommendation
    - One-hot encoding -> similarity
- Embedding-based Recommendation
    - Prod2Vec
- Association-based Recommendation
    - AR (Association Rule)
    - SR (Sequential Rule)
    - MC (Markov Chain)
- Appendix
    - Pandas Basic Tutorial
    - Text Preprocessing
    - Word Representation

### **자주 쓰는 단축키**<br>
Ctrl + Enter: 셀 실행<br>
Shift + Enter: 셀 실행후, 다음 셀로 이동<br>

마크다운 (Markdown)을 수정하려면 더블클릭해주시면 됩니다.

## 기본 패키지 import

In [ ]:
# 기본 패키지 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import warnings

import random

# colab에서 나오는 warning들을 무시합니다.
warnings.filterwarnings('ignore')

# 결과 재현을 위해 해당 코드에서 사용되는 라이브러리들의 Seed를 고정합니다.
def seed_everything(random_seed):
    np.random.seed(random_seed)
    random.seed(random_seed)

seed = 2021
seed_everything(seed)

# 콘텐츠 기반 추천 모델 실습
- One-hot 벡터를 이용한 영화 추천

## 네이버 영화 Dataset 다운로드

In [ ]:
import pandas as pd

from google_drive_downloader import GoogleDriveDownloader as gdd
# https://drive.google.com/file/d/1_HFBNRk-FUOO1nquQVfWbD1IiqnWNzOW/view?usp=sharing
gdd.download_file_from_google_drive(
    file_id='1_HFBNRk-FUOO1nquQVfWbD1IiqnWNzOW',
    dest_path='./naver_movie_dataset.csv',
)

print("데이터 다운로드 완료!")

데이터 다운로드 완료!


In [ ]:
# 데이터셋 불러오기
"""
# of users: 2045, # of movies: 588, # of ratings: 51003
"""

column_names = ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'people', 'country', 'genre']
movie_data = pd.read_csv("naver_movie_dataset.csv", names=column_names)
movie_data.head()

,user_id,item_id,rating,timestamp,title,people,country,genre
0,0,2,7,1494128040,빽 투 더 퓨쳐 2,"['마이클 J. 폭스', '크리스토퍼 로이드', '리 톰슨', '토머스 F. 윌슨'...",['미국'],"['SF', '코미디']"
1,0,3,7,1467529800,빽 투 더 퓨쳐 3,"['마이클 J. 폭스', '크리스토퍼 로이드', '메리 스틴버겐', '토머스 F. ...",['미국'],"['서부', 'SF', '판타지', '코미디']"
2,0,16,9,1513344120,이티,"['헨리 토마스', '디 월리스', '피터 코요테', '로버트 맥노튼', '드류 베...",['미국'],"['판타지', 'SF', '모험', '가족']"
3,0,19,9,1424497980,록키,"['실베스터 스탤론', '탈리아 샤이어', '버트 영', '칼 웨더스', '버제스 ...",['미국'],"['드라마', '액션']"
4,0,20,7,1427627340,록키 2,"['실베스터 스탤론', '탈리아 샤이어', '버트 영', '칼 웨더스', '버제스 ...",['미국'],"['드라마', '액션']"


In [ ]:
# 좀 더 보기 편하게 column 재구성
movie_data = movie_data[['user_id', 'item_id', 'title', 'genre', 'country', 'people']]
movie_data.head()

,user_id,item_id,title,genre,country,people
0,0,2,빽 투 더 퓨쳐 2,"['SF', '코미디']",['미국'],"['마이클 J. 폭스', '크리스토퍼 로이드', '리 톰슨', '토머스 F. 윌슨'..."
1,0,3,빽 투 더 퓨쳐 3,"['서부', 'SF', '판타지', '코미디']",['미국'],"['마이클 J. 폭스', '크리스토퍼 로이드', '메리 스틴버겐', '토머스 F. ..."
2,0,16,이티,"['판타지', 'SF', '모험', '가족']",['미국'],"['헨리 토마스', '디 월리스', '피터 코요테', '로버트 맥노튼', '드류 베..."
3,0,19,록키,"['드라마', '액션']",['미국'],"['실베스터 스탤론', '탈리아 샤이어', '버트 영', '칼 웨더스', '버제스 ..."
4,0,20,록키 2,"['드라마', '액션']",['미국'],"['실베스터 스탤론', '탈리아 샤이어', '버트 영', '칼 웨더스', '버제스 ..."


In [ ]:
# 전체 데이터셋의 user, item 수 확인
user_list = list(movie_data['user_id'].unique())
item_list = list(movie_data['item_id'].unique())
num_users = len(user_list)
num_items = len(item_list)
print(f"# of users: {num_users}, # of items: {num_items}")

# of users: 2045, # of items: 588


## Train, test set 나누기

In [ ]:
# train, test set 나누기
from sklearn.model_selection import train_test_split

train, test = train_test_split(movie_data, test_size=0.2, stratify = movie_data['user_id'], random_state = 1234)

## 데이터에 있는 장르, 국가, 배우 종류 수집하기
- One-hot vector를 구성해주기 위함.

In [ ]:
# 전체 데이터셋을 돌면서 모든 종류의 영화 장르, 국가, 배우 확인
import ast

# genre, country, people
selected_features = ["genre", "country", "people"]
all_genre_list = []
all_country_list = []
all_people_list = []

for index, row in train.iterrows():
    genres = row["genre"]
    coutries = row["country"]
    people = row["people"]
    genres = ast.literal_eval(genres)
    coutries = ast.literal_eval(coutries)
    people = ast.literal_eval(people)
    for genre in genres:
        if genre not in all_genre_list:
            all_genre_list.append(genre)
    for country in coutries:
        if country not in all_country_list:
            all_country_list.append(country)
    for person in people:
        if person not in all_people_list:
            all_people_list.append(person)

num_genres = len(all_genre_list)
num_countries = len(all_country_list)
num_people = len(all_people_list)
print(all_genre_list)
print("# of genres: ", num_genres)
print("# of countries: ", num_countries)
print("# of people: ", num_people)

['드라마', '판타지', '멜로/로맨스', '액션', '공포', '가족', '모험', '코미디', '미스터리', 'SF', '스릴러', '범죄', '서부', '전쟁', '느와르', '뮤지컬', '애니메이션', '서사', '에로']
# of genres:  19
# of countries:  27
# of people:  3948


## One-hot encoding (one-hot vector 생성)

In [ ]:
# one-hot encoding
def binary(feature_list, all_feature_list):
    binary_list = []
    for feature in all_feature_list:
        if feature in feature_list:
            binary_list.append(1)
        else:
            binary_list.append(0)
    
    return binary_list

# genre, country, people feature
train['genre_bin'] = train['genre'].apply(lambda x: binary(x, all_genre_list))
train['country_bin'] = train['country'].apply(lambda x: binary(x, all_country_list))
train['people_bin'] = train['people'].apply(lambda x: binary(x, all_people_list))

# train[['user_id', 'item_id', 'title', 'genre_bin']].head()
train[['user_id', 'item_id', 'title', 'genre_bin', 'country_bin', 'people_bin']].head()

,user_id,item_id,title,genre_bin,country_bin,people_bin
13978,290,564,베어,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
35370,1008,435,천녀유혼,"[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
34053,952,513,영구와 땡칠이,"[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, ..."
5994,79,144,사랑과 영혼,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
35314,1005,31,인디아나 존스 - 최후의 성전,"[0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
# 모든 user에 대해 combined one-hot vector 구하기
# 영화 장르 (genre)에 대해서만 확인해보기
grouped_sum = train['genre_bin'].groupby(by=train['user_id']).sum()

user_bin = {}
for user_idx in user_list:
    total_bin = np.zeros(num_genres)
    num_movies = int(len(grouped_sum[user_idx])/num_genres)

    for i in range(num_movies):
        one_movie = np.array(grouped_sum[user_idx][i*num_genres:(i+1)*num_genres])
        zipped_lists = zip(total_bin, one_movie)
        total_bin = [x + y for (x, y) in zipped_lists]

    total_bin = np.array(total_bin)
    user_bin[user_idx] = (total_bin, num_movies)

In [ ]:
# 특정 user의 one-hot vector 확인해보기
user_id = 10
total_bin = user_bin[user_id][0]
num_movies = user_bin[user_id][1]
print(f"# of movies watched by user {user_id}: {num_movies}")
print("one-hot vector:", total_bin)
print("normalized one-hot vector:", total_bin / num_movies)

## Cosine similarity 계산

In [ ]:
# combined one-hot vector를 가지고 다른 item들과의 cosine similarity 계산
norm_bin = total_bin / num_movies

# unique item 추리기
unique_items = train[['item_id', 'title', 'genre', 'genre_bin', 'country', ]].drop_duplicates(['item_id'])

# 특정 user가 본 영화들 제외
train_items_by_user = train.loc[train.user_id==user_id]
unique_items = unique_items[~unique_items['item_id'].isin(train_items_by_user['item_id'])]

unique_items['similarity'] = unique_items['genre_bin'].apply(lambda x: np.array(x).dot(norm_bin) / (np.array(x).sum() + 1e-10))
unique_items.head()

# cosine similarity를 토대로 top-k item 구하기
sorted_items = unique_items.sort_values(by=['similarity'], axis=0, ascending=False)
sorted_items.head()

,item_id,title,genre,genre_bin,country,similarity
13978,564,베어,['드라마'],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['프랑스'],0.622222
9990,74,챔프,['드라마'],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['미국'],0.622222
22262,522,칠수와 만수,['드라마'],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['한국'],0.622222
20830,515,밤 그리고 도시,['드라마'],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['미국'],0.622222
1532,530,칼리굴라,['드라마'],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","['미국', '이탈리아']",0.622222


In [ ]:
top_k = 10
top_k_items = list(sorted_items['item_id'][:top_k])
top_item_df = sorted_items[['item_id', 'title', 'genre']].drop_duplicates(['item_id'])
# 예측한 top-k items
top_item_df[top_item_df['item_id'].isin(top_k_items[:top_k])]

,item_id,title,genre
13978,564,베어,['드라마']
9990,74,챔프,['드라마']
22262,522,칠수와 만수,['드라마']
20830,515,밤 그리고 도시,['드라마']
1532,530,칼리굴라,['드라마']
55874,465,집시의 시간,['드라마']
36287,439,제7의 봉인,['드라마']
25068,545,깊은 밤 깊은 곳에,['드라마']
27404,48,라 밤바,['드라마']
24369,500,김의 전쟁,['드라마']


In [ ]:
# user가 실제로 본 영화들
user_id = 10
test_items_by_user = test.loc[test.user_id==user_id]
test_items_by_user[['user_id', 'item_id', 'title', 'genre']]

,user_id,item_id,title,genre
486,10,513,영구와 땡칠이,"['가족', '모험', '코미디']"
464,10,192,늑대와 춤을,"['드라마', '서부', '모험']"
419,10,66,티파니에서 아침을,"['멜로/로맨스', '드라마']"
390,10,10,다이 하드,"['액션', '스릴러', '범죄']"
392,10,14,나 홀로 집에,"['코미디', '가족', '모험', '범죄']"
477,10,373,아비정전,"['멜로/로맨스', '드라마', '범죄']"
385,10,1,빽 투 더 퓨쳐,"['SF', '코미디']"
431,10,87,카사블랑카,"['멜로/로맨스', '드라마']"
470,10,245,스카페이스,"['드라마', '액션', '범죄']"
402,10,35,플래툰,"['드라마', '전쟁']"


## Precision, Recall, NDCG@K를 통한 평가

In [ ]:
import math

# evaluation metrices: Precision, Recall, NDCG@K
def compute_metrics(pred_u, target_u, top_k):
    pred_k = pred_u[:top_k]
    num_target_items = len(target_u)

    hits_k = [(i + 1, item) for i, item in enumerate(pred_k) if item in target_u]
    # print("실제로 맞춘 items (position, idx):", hits_k)
    num_hits = len(hits_k)

    idcg_k = 0.0
    for i in range(1, min(num_target_items, top_k) + 1):
        idcg_k += 1 / math.log(i + 1, 2)

    dcg_k = 0.0
    for idx, item in hits_k:
        dcg_k += 1 / math.log(idx + 1, 2)
    
    prec_k = num_hits / top_k
    recall_k = num_hits / min(num_target_items, top_k)
    ndcg_k = dcg_k / idcg_k

    return prec_k, recall_k, ndcg_k

In [ ]:
# user 한 명에 대한 평가
top_k = 200
pred_u = list(sorted_items['item_id'])
target_u = list(test_items_by_user['item_id'])

prec, recall, ndcg = compute_metrics(pred_u, target_u, top_k)
print(f"Precison@{top_k}: {prec:.4f}")
print(f"Recall@{top_k}: {recall:.4f}")
print(f"NDCG@{top_k}: {ndcg:.4f}")

Precison@200: 0.0200
Recall@200: 0.3333
NDCG@200: 0.1108


In [ ]:
# 전체 user에 대한 평가
top_k = 200
prec_list = []
recall_list = []
ndcg_list = []

# unique item 추리기
ori_unique_items = train[['item_id', 'title', 'genre', 'genre_bin', 'country', ]].drop_duplicates(['item_id'])

for user_id in user_list:
    total_bin = user_bin[user_id][0]
    num_movies = user_bin[user_id][1]

    # combined one-hot vector를 가지고 다른 item들과의 cosine similarity 계산
    norm_bin = total_bin / num_movies

    # 특정 user가 본 영화들 제외
    train_items_by_user = train.loc[train.user_id==user_id]
    unique_items = ori_unique_items[~ori_unique_items['item_id'].isin(train_items_by_user['item_id'])]
    unique_items['similarity'] = unique_items['genre_bin'].apply(lambda x: np.array(x).dot(norm_bin) / (np.array(x).sum() + 1e-10))

    # cosine similarity를 토대로 top-k item 구하기
    sorted_items = unique_items.sort_values(by=['similarity'], axis=0, ascending=False)

    test_items_by_user = test.loc[test.user_id==user_id]
    pred_u = list(sorted_items['item_id'])
    target_u = list(test_items_by_user['item_id'])

    prec, recall, ndcg = compute_metrics(pred_u, target_u, top_k)
    prec_list.append(prec)
    recall_list.append(recall)
    ndcg_list.append(ndcg)

In [ ]:
print(f"Precision@{top_k}: {np.mean(prec_list):.4f}")
print(f"Recall@{top_k}: {np.mean(recall_list):.4f}")
print(f"NDCG@{top_k}: {np.mean(ndcg_list):.4f}")

Precision@200: 0.0098
Recall@200: 0.3929
NDCG@200: 0.1062


# 콘텐츠 기반 추천 - 여러 개의 feature 결합
예시로 보여드린 장르 정보와 국가 (country) 혹은 배우 (people) 정보를 활용하여 영화 추천 <br>
- genre + country
- genre + people
- genre + country + people <br>
두 개 이상의 정보를 이용해서 하나의 one-hot vector를 만들 때, 아래처럼 + operator로 두 개의 one-hot vector를 concat해서 one-hot vector를 하나로 만들어주실 수 있습니다.

In [ ]:
# 예시로 genre_bin와 people 두 개를 합치는 방식 사용.

new_feature = 'genre_county'
# new_feature = 'genre_people'
# new_feature = 'genre_people_country'
train[new_feature] = train['genre_bin'] + train['country_bin']
# train[new_feature] = train['genre_bin'] + train['people_bin']
# train[new_feature] = train['genre_bin'] + train['people_bin'] + train['country_bin']

# one-hot vector 길이 변화 확인
print("length of genre:", len(train['genre_bin'].iloc[0]))
print(f"length of {new_feature}:", len(train[new_feature].iloc[0]))

length of genre: 19
length of genre_people_country: 3994


In [ ]:
# 모든 user에 대해 combined one-hot vector 구하기
import numpy as np

grouped_sum = train[new_feature].groupby(by=train['user_id']).sum()
num_features = len(train[new_feature].iloc[0])

user_bin = {}
for user_idx in user_list:
    total_bin = np.zeros(num_features)
    num_dim = int(len(grouped_sum[user_idx])/num_features)

    for i in range(num_dim):
        one_movie = np.array(grouped_sum[user_idx][i*num_features:(i+1)*num_features])
        zipped_lists = zip(total_bin, one_movie)
        total_bin = [x + y for (x, y) in zipped_lists]

    total_bin = np.array(total_bin)
    user_bin[user_idx] = (total_bin, num_dim)

In [ ]:
# 특정 user의 one-hot vector 확인해보기
user_id = 10
total_bin = user_bin[user_id][0]
num_dim = user_bin[user_id][1]
print("one-hot vector:", total_bin)
print("normalized one-hot vector:", total_bin / num_dim)

one-hot vector: [28.  4. 11. ...  0.  0.  0.]
normalized one-hot vector: [0.62222222 0.08888889 0.24444444 ... 0.         0.         0.        ]


## Cosine similarity 계산

In [ ]:
# combined one-hot vector를 가지고 다른 item들과의 cosine similarity 계산
norm_bin = total_bin / num_dim

# unique item 추리기
unique_items = train[['item_id', 'title', 'genre', 'people', 'country', new_feature]].drop_duplicates(['item_id'])

# 특정 user가 본 영화들 제외
train_items_by_user = train.loc[train.user_id==user_id]
unique_items = unique_items[~unique_items['item_id'].isin(train_items_by_user['item_id'])]

unique_items['similarity'] = unique_items[new_feature].apply(lambda x: np.array(x).dot(norm_bin) / (np.array(x).sum() + 1e-10))
unique_items.head()

# cosine similarity를 토대로 top-k item 구하기
sorted_items = unique_items.sort_values(by=['similarity'], axis=0, ascending=False)
sorted_items.head()

,item_id,title,genre,people,country,genre_people_country,similarity
35540,240,살인광 시대,"['드라마', '코미디']","['찰리 채플린', '마사 레이']",['미국'],"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",0.337778
37617,560,로드 하우스,"['드라마', '액션']","['패트릭 스웨이지', '벤 가자라', '샘 엘리어트']",['미국'],"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.288889
48175,159,레드 소냐,[],"['아놀드 슈왈제네거', '브리짓 닐슨']",['미국'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.281481
15269,511,백경,"['드라마', '모험']","['그레고리 펙', '리차드 베이스하트', '리오 겐']",['미국'],"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.262963
41617,198,미지와의 조우,"['SF', '드라마', '모험']","['리차드 드레이퓨즈', '프랑수아 트뤼포']","['미국', '영국']","[1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.250794


In [ ]:
# user가 실제로 본 영화들
user_id = 10
test_items_by_user = test.loc[test.user_id==user_id]
test_items_by_user[['user_id', 'item_id', 'title', 'genre']]

,user_id,item_id,title,genre
486,10,513,영구와 땡칠이,"['가족', '모험', '코미디']"
464,10,192,늑대와 춤을,"['드라마', '서부', '모험']"
419,10,66,티파니에서 아침을,"['멜로/로맨스', '드라마']"
390,10,10,다이 하드,"['액션', '스릴러', '범죄']"
392,10,14,나 홀로 집에,"['코미디', '가족', '모험', '범죄']"
477,10,373,아비정전,"['멜로/로맨스', '드라마', '범죄']"
385,10,1,빽 투 더 퓨쳐,"['SF', '코미디']"
431,10,87,카사블랑카,"['멜로/로맨스', '드라마']"
470,10,245,스카페이스,"['드라마', '액션', '범죄']"
402,10,35,플래툰,"['드라마', '전쟁']"


## Precision, Recall, NDCG@K를 통한 평가

In [ ]:
# user 한 명에 대한 평가
top_k = 200
pred_u = list(sorted_items['item_id'])
target_u = list(test_items_by_user['item_id'])

prec, recall, ndcg = compute_metrics(pred_u, target_u, top_k)
print(f"Precision@{top_k}: {prec:.4f}")
print(f"Recall@{top_k}: {recall:.4f}")
print(f"NDCG@{top_k}: {ndcg:.4f}")

Precision@200: 0.0300
Recall@200: 0.5000
NDCG@200: 0.2409


In [ ]:
# 전체 user에 대한 평가
top_k = 200
prec_list = []
recall_list = []
ndcg_list = []

# unique item 추리기
ori_unique_items = train[['item_id', 'title', 'genre', 'genre_bin', 'country', new_feature]].drop_duplicates(['item_id'])

for user_id in user_list:
    total_bin = user_bin[user_id][0]
    num_movies = user_bin[user_id][1]

    # combined one-hot vector를 가지고 다른 item들과의 cosine similarity 계산
    norm_bin = total_bin / num_movies

    # 특정 user가 본 영화들 제외
    train_items_by_user = train.loc[train.user_id==user_id]
    unique_items = ori_unique_items[~ori_unique_items['item_id'].isin(train_items_by_user['item_id'])]
    unique_items['similarity'] = unique_items[new_feature].apply(lambda x: np.array(x).dot(norm_bin) / (np.array(x).sum() + 1e-10))

    # cosine similarity를 토대로 top-k item 구하기
    sorted_items = unique_items.sort_values(by=['similarity'], axis=0, ascending=False)

    test_items_by_user = test.loc[test.user_id==user_id]
    pred_u = list(sorted_items['item_id'])
    target_u = list(test_items_by_user['item_id'])

    prec, recall, ndcg = compute_metrics(pred_u, target_u, top_k)
    prec_list.append(prec)
    recall_list.append(recall)
    ndcg_list.append(ndcg)

In [ ]:
print(f"Precision@{top_k}: {np.mean(prec_list):.4f}")
print(f"Recall@{top_k}: {np.mean(recall_list):.4f}")
print(f"NDCG@{top_k}: {np.mean(ndcg_list):.4f}")

Precision@200: 0.0128
Recall@200: 0.5074
NDCG@200: 0.1560


# 임베딩 기반 추천 모델 실습
- Prod2Vec

## 네이버 영화 Dataset 다운로드

In [ ]:
import pandas as pd

from google_drive_downloader import GoogleDriveDownloader as gdd
# https://drive.google.com/file/d/1_HFBNRk-FUOO1nquQVfWbD1IiqnWNzOW/view?usp=sharing
gdd.download_file_from_google_drive(
    file_id='1_HFBNRk-FUOO1nquQVfWbD1IiqnWNzOW',
    dest_path='./naver_movie_dataset.csv',
)

print("데이터 다운로드 완료!")

데이터 다운로드 완료!


In [ ]:
# 데이터셋 불러오기
"""
# of users: 2045, # of movies: 588, # of ratings: 51003
"""

column_names = ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'people', 'country', 'genre']
movie_data = pd.read_csv("naver_movie_dataset.csv", names=column_names)
movie_data.head()

,user_id,item_id,rating,timestamp,title,people,country,genre
0,0,2,7,1494128040,빽 투 더 퓨쳐 2,"['마이클 J. 폭스', '크리스토퍼 로이드', '리 톰슨', '토머스 F. 윌슨'...",['미국'],"['SF', '코미디']"
1,0,3,7,1467529800,빽 투 더 퓨쳐 3,"['마이클 J. 폭스', '크리스토퍼 로이드', '메리 스틴버겐', '토머스 F. ...",['미국'],"['서부', 'SF', '판타지', '코미디']"
2,0,16,9,1513344120,이티,"['헨리 토마스', '디 월리스', '피터 코요테', '로버트 맥노튼', '드류 베...",['미국'],"['판타지', 'SF', '모험', '가족']"
3,0,19,9,1424497980,록키,"['실베스터 스탤론', '탈리아 샤이어', '버트 영', '칼 웨더스', '버제스 ...",['미국'],"['드라마', '액션']"
4,0,20,7,1427627340,록키 2,"['실베스터 스탤론', '탈리아 샤이어', '버트 영', '칼 웨더스', '버제스 ...",['미국'],"['드라마', '액션']"


In [ ]:
# 좀 더 보기 편하게 column 재구성
movie_data = movie_data[['user_id', 'item_id', 'rating', 'timestamp', 'title']]
# gensim word2vec 함수에 넣기 위해선 string 형식으로 변형 필요
movie_data['item_id'] = movie_data['item_id'].astype(str)
movie_data.head()

,user_id,item_id,rating,timestamp,title
0,0,2,7,1494128040,빽 투 더 퓨쳐 2
1,0,3,7,1467529800,빽 투 더 퓨쳐 3
2,0,16,9,1513344120,이티
3,0,19,9,1424497980,록키
4,0,20,7,1427627340,록키 2


In [ ]:
# movie title와 id를 매핑할 dictionary를 생성
idx2title = {}
for i, c in zip(movie_data['item_id'], movie_data['title']): 
    idx2title[i] = c

# id와 movie title를 매핑할 dictionary를 생성
title2idx = {}
for i, c in zip(movie_data['item_id'], movie_data['title']): 
    title2idx[c] = i

In [ ]:
# 전체 데이터셋의 user, item 수 확인
user_list = list(movie_data['user_id'].unique())
item_list = list(movie_data['item_id'].unique())
num_users = len(user_list)
num_items = len(item_list)
print(f"# of users: {num_users}, # of items: {num_items}")

# of users: 2045, # of items: 588


## Train, test set 나누기

In [ ]:
# train, test set 나누기
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(movie_data, test_size=0.2, stratify = movie_data['user_id'], random_state = 1234)

## Interaction matrix 형식으로 변형
Movie rating을 implicit feedback으로 여김.

In [ ]:
# train set에 있는 user set
train_users = train_df['user_id'].unique()
# sorting
train_users = sorted(train_users)

train = []
for user_id in train_users:
    itemset = train_df[train_df['user_id'] == user_id]['item_id'].tolist()
    train.append(itemset)
    # print(itemset)

In [ ]:
# test set에서도 동일하게 구성
test_users = test_df['user_id'].unique()
# sorting
test_users = sorted(test_users)

test = []
for user_id in test_users:
    itemset = test_df[test_df['user_id'] == user_id]['item_id'].tolist()
    test.append(itemset)

## Word2Vec으로 embeddings 생성
- https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
from gensim.models import Word2Vec
# CBoW: sg=0
# Skip-gram: sg=1
model = Word2Vec(window=260, sg=1, seed=2021)

In [ ]:
# vocabulary 구성
model.build_vocab(train)

# word2vec 학습
model.train(train, total_examples=model.corpus_count, epochs=20) # total_examples=model.corpus_count

(560631, 816040)

In [ ]:
# 영화 전체 리스트
print(list(title2idx.keys()))

# 특정 영화의 word embedding 확인
movie_title = '이티'
print(f'영화 "{movie_title}"의 item_id: {title2idx[movie_title]}')
# 모든 영화들에 대해서 다음과 같은 word embedding을 생성
model.wv['16']

['빽 투 더 퓨쳐 2', '빽 투 더 퓨쳐 3', '이티', '록키', '록키 2', '록키 3', '록키 4', '록키 5', '터미네이터', '죠스', '에이리언 2', '플래툰', '스팅', '죽은 시인의 사회', '람보', '람보 2', '십계', '언터처블', '황야의 무법자', '대부', '대부 2', '티파니에서 아침을', '토요일 밤의 열기', '러브 스토리', '고스트버스터즈', '고스트버스터즈 2', '탑건', '로마의 휴일', '사관과 신사', '아웃 오브 아프리카', '영웅본색', '라붐', '디어 헌터', '귀여운 여인', '폴링 인 러브', '사랑과 영혼', '로미오와 줄리엣', '터미네이터 2:오리지널', '칵테일', '컬러 오브 머니', '엠마뉴엘', '로보캅', '로보캅 2', '레인 맨', '내츄럴', '해리가 샐리를 만났을 때', '델마와 루이스', '뜨거운 것이 좋아', '스카페이스', '아라비아의 로렌스', '황야의 7인', '브룩클린으로 가는 마지막 비상구', '플래시댄스', '에프 엑스', '해바라기', '성룡의 미라클', '졸업', '차타레 부인의 사랑', '미드나잇 런', '미져리', '여학생', '지옥의 묵시록', '양들의 침묵', '태양은 가득히', '아비정전', '대부 3', '웨스트 사이드 스토리', '지붕 위의 바이올린', '캠퍼스 군단', '크레이머 대 크레이머', '7월 4일생', '특전 유보트', '파리에서의 마지막 탱고', '원초적 본능', '폴리스 스토리', nan, '토탈 리콜', '우묵배미의 사랑', '베로니카의 이중 생활', '엠마뉴엘 3 - 굿바이 엠마뉴엘', '피고인', '누가 로져 래빗을 모함했나', '투 문 정션', '원스 어폰 어 타임 인 아메리카', '분노의 역류', '시네마 천국', '죠스 2', '그렘린', '프레데터', '프레데터 2', '13일의 금요일', '늑대와 춤을', '가위손', '블레이드 러너', '스타워즈 에피소드 4 - 새로운 희망', '스타워즈 에피소드 5 -

array([ 0.04281232, -0.04317594,  0.14327165, -0.08081168,  0.08056643,
       -0.09194097, -0.05745801,  0.21727721,  0.05052005,  0.10638651,
       -0.10162892, -0.0049228 , -0.09344359, -0.12061207,  0.16668229,
       -0.17037693, -0.0997839 , -0.18003686, -0.02929765,  0.18518132,
        0.0545419 ,  0.04112897,  0.00846379, -0.28931582, -0.24444336,
        0.09076742, -0.05354109, -0.20915654, -0.07483517, -0.13917358,
       -0.02949229, -0.2573437 , -0.22061172, -0.06943464, -0.28153116,
       -0.0902845 , -0.25774255, -0.1313576 , -0.00522048, -0.10679169,
       -0.13445112, -0.1771257 ,  0.00251107,  0.11551886,  0.10233859,
        0.05326734, -0.2232377 , -0.14822614, -0.03939514,  0.01198506,
        0.01219224, -0.12387485,  0.07103778,  0.05230504, -0.08538476,
        0.26911637,  0.29993787,  0.20450963,  0.43828613,  0.12134103,
        0.01512854, -0.04205042,  0.18994302,  0.20904493,  0.09929466,
       -0.07738757, -0.12921491, -0.16833441, -0.03807393,  0.02

## 해당 영화와 비슷한 영화 확인

In [ ]:
movie_title = '이티'
sim_movies = model.wv.most_similar(title2idx[movie_title], topn=10)
print('================')
print('영화, similarity')
print('================')
for i, (idx, _) in enumerate(sim_movies):
    print(f'{idx2title[idx]}, {sim_movies[i][1]:.4f}')

영화, similarity
미져리, 0.6795
아마데우스, 0.6766
토탈 리콜, 0.6642
벤허, 0.6640
사랑과 영혼, 0.6631
양들의 침묵, 0.6577
빽 투 더 퓨쳐, 0.6509
바람과 함께 사라지다, 0.6471
죽은 시인의 사회, 0.6469
터미네이터, 0.6438


## 여러 개의 영화를 가지고 추천

In [ ]:
# 사용자가 본 영화들의 embedding의 평균 계산
def aggregate_vectors(movie_list):
    product_vec = []
    for i in movie_list:
        try:
            product_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(product_vec, axis=0)

In [ ]:
user_idx = 10
movie_list = train[user_idx]
target_u = test[user_idx]
print(movie_list)
# 특정 user의 average word embedding
avg_emb = aggregate_vectors(movie_list)
avg_emb.shape
avg_emb

['353', '192', '0', '252', '196', '36', '474', '165', '90', '88', '343', '6', '325', '107', '10', '167', '513', '373', '43', '55', '26', '124', '83', '35', '94', '3', '184', '483', '14', '41', '480', '120', '315', '405', '64', '236', '2', '18', '587', '44', '19', '82', '442', '81', '418']


array([ 0.03856993, -0.0961246 ,  0.09010831,  0.00995466,  0.03412318,
       -0.06117509, -0.09824856,  0.1284571 , -0.01692738, -0.05303491,
       -0.10091333, -0.03992696, -0.07363348, -0.08581712,  0.12302535,
       -0.13707685, -0.14919117, -0.12231461,  0.08999959,  0.09863811,
       -0.13792337,  0.04791999,  0.01710796, -0.13386646, -0.18302943,
        0.11230587,  0.00801678, -0.30733883,  0.05668146, -0.04092204,
       -0.1473755 , -0.15037891, -0.20005646,  0.02946776, -0.0568874 ,
       -0.04843221, -0.13045739, -0.12622413,  0.10136108,  0.04673307,
       -0.12729508, -0.18372285,  0.08597168,  0.06479196,  0.20119265,
        0.11142042, -0.07119346, -0.11229873, -0.03582325, -0.0252181 ,
       -0.03773727, -0.03072629,  0.07724961,  0.02219275, -0.03173674,
        0.2894631 ,  0.11283343,  0.10550309,  0.23908533, -0.01123946,
        0.10467631, -0.19198288,  0.30330604,  0.0747011 ,  0.240452  ,
       -0.05292048, -0.13169038, -0.11594621, -0.1191218 ,  0.02

In [ ]:
sim_movies_all = model.similar_by_vector(avg_emb, topn=300)
# train에 없는 영화만 남기기
sim_movies = []
for i, _ in enumerate(sim_movies_all):
    if sim_movies_all[i][0] not in movie_list:
        sim_movies.append((sim_movies_all[i][0], sim_movies_all[i][1]))
print('=====================')
print('영화, idx, similarity')
print('=====================')
# top-N movie list 반환
pred_u = []
for i, (idx, _) in enumerate(sim_movies):
    pred_u.append(sim_movies[i][0])
    print(f'{idx2title[idx]}, {sim_movies[i][0]}, {sim_movies[i][1]:.4f}')
    if i == 9:
        break

영화, idx, similarity
레인 맨, 188, 0.7931
가위손, 561, 0.7922
사랑과 영혼, 144, 0.7829
빽 투 더 퓨쳐, 1, 0.7809
터미네이터 2:오리지널, 164, 0.7785
이티, 16, 0.7756
델마와 루이스, 203, 0.7735
미션, 70, 0.7733
배트맨, 199, 0.7663
원스 어폰 어 타임 인 아메리카, 575, 0.7650


## Precision, Recall, NDCG@K를 통한 평가

In [ ]:
import math

# evaluation metrices: Precision, Recall, NDCG@K
def compute_metrics(pred_u, target_u, top_k):
    pred_k = pred_u[:top_k]
    num_target_items = len(target_u)
    
    hits_k = [(i + 1, item) for i, item in enumerate(pred_k) if item in target_u]
    # print("실제로 맞춘 items (position, idx):", hits_k)
    num_hits = len(hits_k)

    idcg_k = 0.0
    for i in range(1, min(num_target_items, top_k) + 1):
        idcg_k += 1 / math.log(i + 1, 2)

    dcg_k = 0.0
    for idx, item in hits_k:
        dcg_k += 1 / math.log(idx + 1, 2)
    
    prec_k = num_hits / top_k
    recall_k = num_hits / min(num_target_items, top_k)
    ndcg_k = dcg_k / idcg_k

    return prec_k, recall_k, ndcg_k

In [ ]:
top_k = 200
print(f'모델이 예측한 movies: {pred_u}')
print(f'실제로 본 movies: {target_u}')
# user 한 명에 대한 평가
prec, recall, ndcg = compute_metrics(pred_u, target_u, top_k)
print(f"Precison@{top_k}: {prec:.4f}")
print(f"Recall@{top_k}: {recall:.4f}")
print(f"NDCG@{top_k}: {ndcg:.4}")

모델이 예측한 movies: ['188', '561', '144', '1', '164', '16', '203', '70', '199', '575']
실제로 본 movies: ['97', '190', '87', '334', '561', '575', '1', '164', '66', '245', '91', '20']
Precison@200: 0.0200
Recall@200: 0.3333
NDCG@200: 0.3412


In [ ]:
# 전체 user에 대한 평가
top_k = 200
prec_list = []
recall_list = []
ndcg_list = []

for user_idx in range(num_users):
    movie_list = train[user_idx]
    target_u = test[user_idx]
    avg_emb = aggregate_vectors(movie_list)

    sim_movies_all = model.similar_by_vector(avg_emb, topn=500)
    # train에 없는 영화만 남기기
    sim_movies = []
    for i, _ in enumerate(sim_movies_all):
        if sim_movies_all[i][0] not in movie_list:
            sim_movies.append((sim_movies_all[i][0], sim_movies_all[i][1]))
    # top-N movie list 반환
    pred_u = []
    for i, (idx, _) in enumerate(sim_movies):
        pred_u.append(sim_movies[i][0])

    prec, recall, ndcg = compute_metrics(pred_u, target_u, top_k)
    prec_list.append(prec)
    recall_list.append(recall)
    ndcg_list.append(ndcg)

In [ ]:
print(f"Precision@{top_k}: {np.mean(prec_list):.4f}")
print(f"Recall@{top_k}: {np.mean(recall_list):.4f}")
print(f"NDCG@{top_k}: {np.mean(ndcg_list):.4f}")

Precision@200: 0.0224
Recall@200: 0.9071
NDCG@200: 0.3589


# Association rule (AR), sequential rule (SR), markov chain (MC) 실습

## 기본 패키지 및 함수

In [ ]:
# 기본 패키지 import
import pandas as pd
import numpy as np
import warnings
import random
import math

# colab에서 나오는 warning들을 무시
warnings.filterwarnings('ignore')

# 2d array를 dictionary로 만듦
# input: [[user_id, item_id, timestamp], ...] 형태의 numpy array
# output: {user_id: [item1, item2, ......], ...} 형태의 dictionary
def make_to_dict(data):
    data_dict = {}
    cur_user = -1
    tmp_user = []
    for row in data:
        user_id, item_id = row[0], row[1]
        if user_id != cur_user:
            if cur_user != -1:
                tmp = np.asarray(tmp_user)
                tmp_items = tmp[:,1]
                data_dict[cur_user] = list(tmp_items)
                
            cur_user = user_id
            tmp_user = []
        tmp_user.append(row)

    if cur_user != -1:
        tmp = np.asarray(tmp_user)
        tmp_items = tmp[:,1]
        data_dict[cur_user] = list(tmp_items)
        
    return data_dict


# train set과 test set을 shuffle없이 나눔
# input: [[user_id, item_id, timestamp], ...] 형태의 numpy array
# output
#    - train: [[user_id, item_id, timestamp], ...] 형태의 numpy array
#    - test: [[user_id, item_id, timestamp], ...] 형태의 numpy array
def train_test_split_no_shuffle(movie_data):
    train = []
    test = []
    cur_user = -1
    tmp_user = []
    for row in movie_data:
        user_id, item_id = row[0], row[1]
        if user_id != cur_user:
            if cur_user != -1:
                user_rating_num = len(tmp_user)
                split_index = int(user_rating_num*0.8)
                train.extend(tmp_user[:split_index])
                test.extend(tmp_user[split_index:])

            cur_user = user_id
            tmp_user = []
        tmp_user.append(row)

    if cur_user != -1:
        user_rating_num = len(tmp_user)
        split_index = int(user_rating_num*0.8)
        train.extend(tmp_user[:split_index])
        test.extend(tmp_user[split_index:])

    return np.asanyarray(train), np.asanyarray(test)


# Precision, Recall, NDCG@K 평가
# input
#    - pred_u: 예측 값으로 정렬 된 item index
#    - target_u: test set의 item index
#    - top_k: top-k에서의 k 값
# output: prec_k, recall_k, ndcg_k의 점수
def compute_metrics(pred_u, target_u, top_k):
    pred_k = pred_u[:top_k]
    num_target_items = len(target_u)

    hits_k = [(i + 1, item) for i, item in enumerate(pred_k) if item in target_u]

    num_hits = len(hits_k)

    idcg_k = 0.0
    for i in range(1, min(num_target_items, top_k) + 1):
        idcg_k += 1 / math.log(i + 1, 2)

    dcg_k = 0.0
    for idx, item in hits_k:
        dcg_k += 1 / math.log(idx + 1, 2)
    
    prec_k = num_hits / top_k
    recall_k = num_hits / min(num_target_items, top_k)
    ndcg_k = dcg_k / idcg_k

    return prec_k, recall_k, ndcg_k

## 네이버 영화 Dataset 다운로드

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
# https://drive.google.com/file/d/1_HFBNRk-FUOO1nquQVfWbD1IiqnWNzOW/view?usp=sharing
gdd.download_file_from_google_drive(
    file_id='1_HFBNRk-FUOO1nquQVfWbD1IiqnWNzOW',
    dest_path='./naver_movie_dataset.csv',
)

print("데이터 다운로드 완료!")

데이터 다운로드 완료!


In [ ]:
# 데이터셋 불러오기
column_names = ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'people', 'country', 'genre']
movie_data = pd.read_csv("naver_movie_dataset.csv", names=column_names)

# 전체 데이터셋의 user, item 수 확인
user_list = list(movie_data['user_id'].unique())
item_list = list(movie_data['item_id'].unique())
num_users = len(user_list)
num_items = len(item_list)
num_ratings = len(movie_data)
print(f"# of users: {num_users},  # of items: {num_items},  # of ratings: {num_ratings}")

# of users: 2045,  # of items: 588,  # of ratings: 51003


## 데이터 전처리 및 train, test set 나누기

In [ ]:
# movie title와 id를 매핑할 dict를 생성
item_id_title = movie_data[['item_id', 'title']]
item_id_title = item_id_title.drop_duplicates()

idx2title = {}
for i, c in zip(item_id_title['item_id'], item_id_title['title']): 
    idx2title[i] = c

# train, test set을 timestamp기준으로 나누기
movie_data = movie_data[['user_id', 'item_id', 'timestamp']]
movie_data = movie_data.sort_values(by=["user_id", "timestamp"], ascending=[True, True]) # timestamp로 정렬
movie_data = movie_data.to_numpy()
train, test = train_test_split_no_shuffle(movie_data)

## Association Rule (AR) 구현

In [ ]:
class AssociationRules: 
    def __init__(self):
        self.rules = dict()
            
    def fit(self, data):
        cur_session = -1
        last_items = []
        for row in data:
            session_id, item_id = row[0], row[1]
            
            if session_id != cur_session:
                cur_session = session_id
                last_items = []
            else: 
                for item_id2 in last_items: 
                    
                    if not item_id in self.rules :
                        self.rules[item_id] = dict()
                    
                    if not item_id2 in self.rules :
                        self.rules[item_id2] = dict()
                    
                    if not item_id in self.rules[item_id2]:
                        self.rules[item_id2][item_id] = 0
                    
                    if not item_id2 in self.rules[item_id]:
                        self.rules[item_id][item_id2] = 0
                    
                    self.rules[item_id][item_id2] += 1
                    self.rules[item_id2][item_id] += 1
                    
            last_items.append( item_id )


        # normalization
        cur_session = -1
        normal = dict()
        items = []
        for row in data:
            session_id, item_id = row[0], row[1]
            if session_id != cur_session:
                if len(items) != 0:
                    session_len = len(items) - 1 # |p|-1
                    for item_in_session in items:
                        if not item_in_session in normal:
                            normal[item_in_session] = 0
                        normal[item_in_session] += session_len

                cur_session = session_id
                items = []
            else:
                items.append(item_id)

        session_len = len(items) - 1 # |p|-1
        for item_in_session in items:
            if not item_in_session in normal:
                normal[item_in_session] = 0
            normal[item_in_session] += session_len


        for item_id1 in self.rules.keys():
            if item_id1 in normal:
                normal_term = normal[item_id1]
            else:
                continue

            if normal_term == 0:
                continue

            for item_id2 in self.rules[item_id1].keys():
                self.rules[item_id1][item_id2] = self.rules[item_id1][item_id2] / normal_term


    def predict(self, input_item_id, predict_for_item_ids):
        # input_item_id: session의 마지막 item id
        # predict_for_item_ids: missing feedback (추천 후보 항목들)

        preds = np.zeros(len(predict_for_item_ids))

        if input_item_id in self.rules:
            for i, predicted_item in enumerate(predict_for_item_ids):
                if predicted_item in self.rules[input_item_id]:
                    preds[i] = self.rules[input_item_id][predicted_item]
                else:
                    preds[i] = 0.0

        return preds

## Sequential Rule (SR) 구현

In [ ]:
class SequentialRules: 
    def __init__(self):
        self.rules = dict()
            
    def fit(self, data):
        cur_session = -1
        last_items = []

        for row in data:
            session_id, item_id = row[0], row[1]

            if session_id != cur_session:
                cur_session = session_id
                last_items = []
            else:
                for i in range(1, len(last_items) + 1):
                    prev_item = last_items[-i]

                    if not prev_item in self.rules:
                        self.rules[prev_item] = dict()

                    if not item_id in self.rules[prev_item]:
                        self.rules[prev_item][item_id] = 0

                    weight = 1/i
                    self.rules[prev_item][item_id] += weight

            last_items.append(item_id)


        # normalization
        normal = dict()
        cur_session = -1
        x = 1
        for row in data:
            session_id, item_id = row[0], row[1]
            if session_id != cur_session:
                cur_session = session_id
                x = 1
            else:
                if not item_id in normal:
                    normal[item_id] = 0
                normal[item_id] += x
            x += 1

        for item_id1 in self.rules.keys():
            if item_id1 in normal:
                normal_term = normal[item_id1]
            else:
                continue

            if normal_term == 0:
                continue

            for item_id2 in self.rules[item_id1].keys():
                self.rules[item_id1][item_id2] = self.rules[item_id1][item_id2] / normal_term


    def predict(self, input_item_id, predict_for_item_ids):
        # input_item_id: session의 마지막 item id
        # predict_for_item_ids: missing feedback (추천 후보 항목들)

        preds = np.zeros(len(predict_for_item_ids))

        if input_item_id in self.rules:
            for i, predicted_item in enumerate(predict_for_item_ids):
                if predicted_item in self.rules[input_item_id]:
                    preds[i] = self.rules[input_item_id][predicted_item]
                else:
                    preds[i] = 0.0

        return preds

## Markov Chain (MC) 구현

In [ ]:
class MarkovModel:
    def __init__(self):
        self.rules = dict()

    def fit(self, data):
        cur_session = -1
        prev_item = -1

        for row in data:
            session_id, item_id = row[0], row[1]

            if session_id != cur_session:
                cur_session = session_id
            else:                 
                if not prev_item in self.rules :
                    self.rules[prev_item] = dict()
                
                if not item_id in self.rules[prev_item]:
                    self.rules[prev_item][item_id] = 0
                
                self.rules[prev_item][item_id] += 1
            
            prev_item = item_id


        # normalization
        normal = dict()
        cur_session = -1
        for i in range(0, len(data)-1):
            row = data[i]
            session_id, item_id = row[0], row[1]
            if session_id != cur_session:
                cur_session = session_id
            else:
                if not item_id in normal:
                    normal[item_id] = 0
                normal[item_id] += 1

        for item_id1 in self.rules.keys():
            if item_id1 in normal:
                normal_term = normal[item_id1]
            else:
                continue

            if normal_term == 0:
                continue

            for item_id2 in self.rules[item_id1].keys():
                self.rules[item_id1][item_id2] = self.rules[item_id1][item_id2] / normal_term


    def predict(self, input_item_id, predict_for_item_ids):
        # input_item_id: session의 마지막 item id
        # predict_for_item_ids: missing feedback (추천 후보 항목들)

        preds = np.zeros(len(predict_for_item_ids))

        if input_item_id in self.rules:
            for i, predicted_item in enumerate(predict_for_item_ids):
                if predicted_item in self.rules[input_item_id]:
                    preds[i] = self.rules[input_item_id][predicted_item]
                else:
                    preds[i] = 0.0

        return preds

## 알고리즘 별 규칙 구하기

In [ ]:
# Association rules 구하기
AR = AssociationRules()
AR.fit(train)

# Sequential rules 구하기
SR = SequentialRules()
SR.fit(train)

# Markov chain rules 구하기
MC = MarkovModel()
MC.fit(train)

In [ ]:
# train과 test data를 dict형태로 변환
train_dict = make_to_dict(train)
test_dict = make_to_dict(test)

## 특정 사용자에 대한 정성 평가

In [ ]:
user_id = 1500 # 1005, 1500

# 사용자가 본 최근 영화들 10개
rated_items = train_dict[user_id]
for rated_item in rated_items[-10:]:
    print(idx2title[rated_item])

델마와 루이스
천녀유혼 2 - 인간도
보디 히트
프레데터
프레데터 2
네버엔딩 스토리
나인 하프 위크
복수무정
매드 맥스 3
다이 하드


In [ ]:
# 사용자가 본 마지막 영화
rated_items = train_dict[user_id]
print("User id %d이 본 마지막 영화: "%(user_id), idx2title[ rated_items[-1]])

User id 1500가 본 마지막 영화:  다이 하드


In [ ]:
# 각 알고리즘 별 추천 영화 Top-10

# 사용자가 이미 본 영화들 제외
missing_items = list(set(item_list) - set(train_dict[user_id]))

# 사용자의 마지막 영화를 기준으로 각 알고리즘을 이용한 추천 점수 구함
predicted_scores = []
for algo in [AR, SR, MC]:
    predicted_scores.append(algo.predict(train_dict[user_id][-1], missing_items))

# 추천 점수로 top-N 항목 추천
missing_items = np.asarray(missing_items)

print("각 알고리즘 별 추천 영화 Top-10")
algo_names = ["AR", "SR", "MC"]
for i, predicted_score_algo in enumerate(predicted_scores):
    predicted_score_algo = np.asarray(predicted_score_algo)
    predicted_score_algo_idx = np.argsort(predicted_score_algo)
    predicted_score_algo_idx = predicted_score_algo_idx[::-1]
    ranked_items = missing_items[predicted_score_algo_idx]

    print("==============================")     
    print(algo_names[i] + " results")
    print("==============================")     
    for ranked_item in ranked_items[:10]:
        print(idx2title[ranked_item])
    print("")

각 알고리즘 별 추천 영화 Top-10
AR results
터미네이터 2:오리지널
다이 하드 2
나 홀로 집에
터미네이터
죽은 시인의 사회
대부
빽 투 더 퓨쳐
시네마 천국
영웅본색
에이리언 2

SR results
다이 하드 2
터미네이터
터미네이터 2:오리지널
죽은 시인의 사회
이티
대부
빽 투 더 퓨쳐
빠삐용
나 홀로 집에
록키

MC results
다이 하드 2
빠삐용
이티
죽은 시인의 사회
늑대와 춤을
터미네이터 2:오리지널
터미네이터
록키
스카페이스
대부



In [ ]:
# 사용자가 실제로 본 영화들
rated_items = test_dict[user_id]
for rated_item in rated_items:
    print(idx2title[rated_item])

다이 하드 2
레이더스
인디아나 존스
인디아나 존스 - 최후의 성전
우먼 인 레드
누가 로져 래빗을 모함했나
레드 소냐


## 특정 사용자에 대한 정량 평가

In [ ]:
user_id = 1500

# 사용자가 이미 본 영화들 제외
missing_items = list(set(item_list) - set(train_dict[user_id]))

# 사용자의 마지막 영화를 각 알고리즘으로 항목 추천
predicted_scores = []
for algo in [AR, SR, MC]:
    predicted_scores.append(algo.predict(train_dict[user_id][-1], missing_items))

# 각 알고리즘의 예측값으로 내림차순 정렬된 항목들의 list 구함
missing_items = np.asarray(missing_items)

sorted_missing_items = []
for predicted_score_algo in predicted_scores:
    predicted_score_algo = np.asarray(predicted_score_algo)
    predicted_score_algo_idx = np.argsort(predicted_score_algo)
    predicted_score_algo_idx = predicted_score_algo_idx[::-1]
    sorted_missing_items.append(missing_items[predicted_score_algo_idx])


top_k = 100
test_items = test_dict[user_id]
algo_names = ["AR", "SR", "MC"]
for i, sorted_missing_algo in enumerate(sorted_missing_items):
    print(algo_names[i] + " results" )
    prec, recall, ndcg = compute_metrics(sorted_missing_algo, test_items, top_k)
    print(f"Precision@{top_k}: {prec:.3f}")
    print(f"Recall@{top_k}: {recall:.3f}")
    print(f"NDCG@{top_k}: {ndcg:.3f}")
    print("")


AR results
Precision@100: 0.040
Recall@100: 0.571
NDCG@100: 0.361

SR results
Precision@100: 0.040
Recall@100: 0.571
NDCG@100: 0.451

MC results
Precision@100: 0.030
Recall@100: 0.429
NDCG@100: 0.390



## 전체 사용자에 대한 정량 평가

In [ ]:
# 전체 user에 대한 평가
top_k = 50
algo_names = ["AR", "SR", "MC"]

prec_list = {}
recall_list = {}
ndcg_list = {}
for algo in algo_names:
    prec_list[algo] = []
    recall_list[algo] = []
    ndcg_list[algo] = []

for user_id in user_list:
    missing_items = list(set(item_list) - set(train_dict[user_id]))

    predicted_scores = []
    for algo in [AR, SR, MC]:
        predicted_scores.append(algo.predict(train_dict[user_id][-1], missing_items))

    missing_items = np.asarray(missing_items)

    sorted_missing_items = []
    for predicted_score_algo in predicted_scores:
        predicted_score_algo = np.asarray(predicted_score_algo)
        predicted_score_algo_idx = np.argsort(predicted_score_algo)
        predicted_score_algo_idx = predicted_score_algo_idx[::-1]
        sorted_missing_items.append(missing_items[predicted_score_algo_idx])

    test_items = test_dict[user_id]

    for i, sorted_missing_algo in enumerate(sorted_missing_items):
        prec, recall, ndcg = compute_metrics(sorted_missing_algo, test_items, top_k)
        prec_list[algo_names[i]].append(prec)
        recall_list[algo_names[i]].append(recall)
        ndcg_list[algo_names[i]].append(ndcg)


In [ ]:
for algo in algo_names:
    print(algo + " results" )
    print(f"Precision@{top_k}: {np.mean(prec_list[algo]):.3f}")
    print(f"Recall@{top_k}: {np.mean(recall_list[algo]):.3f}")
    print(f"NDCG@{top_k}: {np.mean(ndcg_list[algo]):.3f}")
    print("")

AR results
Precision@100: 0.056
Recall@100: 0.553
NDCG@100: 0.326

SR results
Precision@100: 0.051
Recall@100: 0.515
NDCG@100: 0.297

MC results
Precision@100: 0.040
Recall@100: 0.421
NDCG@100: 0.251



# Appendix

## Pandas Basic Tutorial

Pandas Dataframe - 2차원 구조의 Data를 쉽게 저장/확인/수정하기 위한 방법으로 사용하는 data format.

### Dataframe 생성

In [ ]:
# pandas DataFrame 생성 기본 명령
# pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

import pandas as pd

df = pd.DataFrame([[1,2,3,4],[5,6,7,8]])
df

,0,1,2,3
0,1,2,3,4
1,5,6,7,8


In [ ]:
# Dictionary 형태의 data로 생성.
dict_data = {'col0': [1,2,3,4],
             'col1': [5,6,7,8],
             'col2': [9,10,11,12],}
row = ['row0','row1','row2', 'row3']

data = pd.DataFrame(data=dict_data, index=row)
data

,col0,col1,col2
row0,1,5,9
row1,2,6,10
row2,3,7,11
row3,4,8,12


In [ ]:
# Array 형태의 data로 생성.
array_data = [[1,2,3,4],
              [5,6,7,8],
              [9,10,11,12]]
row = ['row0','row1','row2']
column = ['col0', 'col1', 'col2', 'col3']

data = pd.DataFrame(data=array_data, index=row, columns=column)
data

,col0,col1,col2,col3
row0,1,2,3,4
row1,5,6,7,8
row2,9,10,11,12


### Dataframe 내의 data 접근/수정

In [ ]:
# column으로 바로 접근
data['col0']

row0    1
row1    5
row2    9
Name: col0, dtype: int64

In [ ]:
# 특정 열에 접근
data.loc['row0',:]

col0    1
col1    2
col2    3
col3    4
Name: row0, dtype: int64

In [ ]:
# 특정 행에 접근
data.loc[:,'col0']

row0    1
row1    5
row2    9
Name: col0, dtype: int64

In [ ]:
# 특정 data에 접근
data.loc['row0','col0']

1

In [ ]:
# 조건문을 통한 접근
data.loc[data['col1'].isin([1,2,3,4,5,6])]  # data['col1'] = [2, 6, 10]

,col0,col1,col2,col3
row0,1,2,3,4
row1,5,6,7,8


In [ ]:
# 특정 data 수정
data.loc['row0','col0'] = 0
data

,col0,col1,col2,col3
row0,0,2,3,4
row1,5,6,7,8
row2,9,10,11,12


## Word Representation

### Dataset: 네이버 뉴스

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
# https://drive.google.com/file/d/1kpFLOEL3yYEPeaoDSYIfkRM0uuRvOcb5/view?usp=sharing
gdd.download_file_from_google_drive(
    file_id='1kpFLOEL3yYEPeaoDSYIfkRM0uuRvOcb5',
    dest_path='./naver_news_dataset.csv',
)

print("데이터 다운로드 완료!")

데이터 다운로드 완료!


In [ ]:
# 2021-06-15 13시 04분에 수집한 데이터셋
dataset = pd.read_csv('./naver_news_dataset.csv')
dataset = dataset.sample(frac=1, random_state=2021).reset_index(drop=True)
dataset_backup = dataset

print('# of News data:',len(dataset))
dataset.head()

# of News data: 2000


,title,press,query,date,url
0,삼성·모더나-SK·노바백스 협력…한미 '백신 동맹' 구체화,노컷뉴스,노바백스,2021.05.23.,https://www.nocutnews.co.kr/news/5557072
1,대구서 얀센 접종 30대 남성 사흘 만에 '의문의 사망',MBN,얀센,1일 전,http://www.mbn.co.kr/pages/news/newsView.php?n...
2,코로나19 어제 399명 신규확진… 77일만에 400명 아래(종합),조선비즈,코로나,1일 전,https://biz.chosun.com/policy/politics/2021/06...
3,"진안군, 코로나 백신 접종자에 체육시설 이용료 최대 80％ 할인",연합뉴스,코로나,1시간 전,http://yna.kr/AKR20210615081700055?did=1195m
4,2천만명분 도입 노바백스 백신 곧 국내 허가 신청할 듯(종합),연합뉴스,노바백스,2021.04.26.,http://yna.kr/AKR20210426068351017?did=1195m


### BoW (Bag of Words)
document 1개에 대해서만 BoW 실행

In [ ]:
document = ['사람이 온다는 건 실은 어마어마한 일이다.']

In [ ]:
# 뉴스 데이터셋
# document = dataset['title'][:1]

# sklearn 사용하여 BoW
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
bow = vect.fit_transform(document).toarray()
word_column = vect.get_feature_names()

In [ ]:
pd.DataFrame(data=bow, columns=word_column)

,사람이,실은,어마어마한,온다는,일이다
0,1,1,1,1,1


### DTM (Document-Term Matrix)
각 document들의 BoW를 하나의 행렬로 나타낸 것

In [ ]:
document = ['사람이 온다는 건 실은 어마어마한 일이다.', '그는 그의 과거와 현재와 그리고 그의 미래와 함께 오기 때문이다.', '한 사람의 일생이 오기 때문이다.']

In [ ]:
# 뉴스 데이터셋
# document = dataset['title'][:5]

vect = CountVectorizer()
BoW = vect.fit_transform(document).toarray()
word_column = vect.get_feature_names()

In [ ]:
# DTM
pd.DataFrame(data=BoW, columns=word_column)

,과거와,그는,그리고,그의,때문이다,미래와,사람의,사람이,실은,어마어마한,오기,온다는,일생이,일이다,함께,현재와
0,0,0,0,0,0,0,0,1,1,1,0,1,0,1,0,0
1,1,1,1,2,1,1,0,0,0,0,1,0,0,0,1,1
2,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0


### TF-IDF (Term Frequency-Inverse Document Frequency)

BoW처럼 단어의 출현 빈도(Term Frequency)만 고려하는 것이 아닌, Inverse Document Frequency도 고려하여 각 단어의 중요 정도를 가중치로 주는 방법.

In [ ]:
document = ['사람이 온다는 건 실은 어마어마한 일이다.', '그는 그의 과거와 현재와 그리고 그의 미래와 함께 오기 때문이다.', '한 사람의 일생이 오기 때문이다.']

In [ ]:
# 뉴스 데이터셋
# document = dataset['title'][:5]

from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_vect = TfidfVectorizer()
Tfidf = Tfidf_vect.fit_transform(document).toarray()
word_column = Tfidf_vect.get_feature_names()

In [ ]:
# DTM (TfIdf applied)
pd.DataFrame(data=Tfidf, columns=word_column)

,과거와,그는,그리고,그의,때문이다,미래와,사람의,사람이,실은,어마어마한,오기,온다는,일생이,일이다,함께,현재와
0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.447214,0.447214,0.447214,0.000000,0.447214,0.000000,0.447214,0.000000,0.000000
1,0.299385,0.299385,0.299385,0.59877,0.227690,0.299385,0.000000,0.000000,0.000000,0.000000,0.227690,0.000000,0.000000,0.000000,0.299385,0.299385
2,0.000000,0.000000,0.000000,0.00000,0.428046,0.000000,0.562829,0.000000,0.000000,0.000000,0.428046,0.000000,0.562829,0.000000,0.000000,0.000000
